In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the dataset
day_df  = pd.read_csv('data/day.csv')
hour_df = pd.read_csv('data/hour.csv')

# Add source column to each dataset
# day_df['source'] = 'day'
# hour_df['source'] = 'hour'

# Merge the datasets
merged_df = pd.concat([day_df, hour_df], ignore_index=True)
merged_df = merged_df.drop(['dteday'], axis=1)

#TEST İÇİN 10 VERİYE İN
# merged_df = merged_df.head(10)

non_numeric_cols = merged_df.select_dtypes(exclude=['int64', 'float64']).columns
if len(non_numeric_cols) > 0:
      merged_df = pd.get_dummies(merged_df, columns=non_numeric_cols, drop_first=True)

merged_df = merged_df.dropna()

X = merged_df.drop('cnt', axis=1).to_numpy()
y = merged_df['cnt'].to_numpy().ravel()

scaler = StandardScaler()
X      = scaler.fit_transform(X)

X.shape, y.shape


((17379, 15), (17379,))

In [3]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import time

# Start runtime measurement
start_time = time.time()

# Initialize the SVM regressor with a linear kernel
svr = SVR(kernel='linear')

# Perform k-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []
r2_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Train the model
    svr.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = svr.predict(X_test)
    
    # Evaluate performance
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    mse_scores.append(mse)
    r2_scores.append(r2)

# Print average performance metrics
print(f'Average Mean Squared Error (MSE): {np.mean(mse_scores):.2f}')
print(f'Average R² Score: {np.mean(r2_scores):.2f}')

# End runtime measurement
end_time = time.time()
print(f'Total Runtime: {end_time - start_time:.2f} seconds')

Average Mean Squared Error (MSE): 0.00
Average R² Score: 1.00
Total Runtime: 1.94 seconds
